![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Vector Search with Redispy
## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/redispy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Packages

In [2]:
# NBVAL_SKIP
%pip install -q redis numpy sentence-transformers


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from redis import Redis

### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [4]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client

In [5]:
client = Redis.from_url(REDIS_URL)

## Prepare data

In this examples we will load a list of movie objects with the following attributes: `title`, `rating`, `description`, and `genre`. 

For the vector part of our vector search we will embed the description so that user's can search for movies that best match what they're looking for.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

In [6]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [37]:
from sentence_transformers import SentenceTransformer

# load model for embedding our movie descriptions
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(model, text):
    return np.array(model.encode(text)).astype(np.float32).tobytes()

/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# Note: convert embedding array to bytes for storage in Redis Hash data type
movie_data = [
    {
        **movie,
        "vector": embed_text(model, movie["description"])
    } for movie in movies
]

In [9]:
movie_data[0]

{'title': 'Explosive Pursuit',
 'genre': 'action',
 'rating': 7,
 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'vector': b'\x00\x00\x00`\xd3\x8c\xaf?\x00\x00\x00\xc0\x01La?\x00\x00\x00@D\xf2v?\x00\x00\x00\x80g\xd9\xaf\xbf\x00\x00\x00@\x9f\xccy\xbf\x00\x00\x00`\xd8B\xa9?\x00\x00\x00\xc0\xea\xf4\xa7?\x00\x00\x00\xa0\xdd\xae\x92?\x00\x00\x00\x80L\xdf\xc0\xbf\x00\x00\x00\x80"\xeb\xb9?\x00\x00\x00\x00\xe5\x00\xae?\x00\x00\x00\xe0g\xbb\xa1\xbf\x00\x00\x00\xa0R\x1e\xa9\xbf\x00\x00\x00@\xad\xcc\x98?\x00\x00\x00\xa0\xe8;\xac?\x00\x00\x00@\x0f\xc7\x82\xbf\x00\x00\x00\x00\x80z\x82?\x00\x00\x00\xc0G\x95\xa3?\x00\x00\x00\xa0\xdf=\x91?\x00\x00\x00\xa0\x17\xb6\x85?\x00\x00\x00@P\xf6\x93\xbf\x00\x00\x00`ka\xb8\xbf\x00\x00\x00\x00\xd3I\xaa?\x00\x00\x00\x00\x8f\xed\x9e\xbf\x00\x00\x00\xc0\xc9\xe7\xc2\xbf\x00\x00\x00`D\xa2 \xbf\x00\x00\x00 \xa7\xee\x97?\x00\x00\x00\x00\x16\\L\xbf\x00\x00\x00`\xda\x14\xb5\xbf\x00\x00\x00\x00\x8f\x9b\x9

## Define Redis index schema

In [43]:
from redis.commands.search.field import VectorField, TagField, NumericField, TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

index_name = "movies"

schema = (
    VectorField(
        "vector",
        "HNSW",
        {
            "TYPE": "FLOAT32",
            "DIM": 384,
            "DISTANCE_METRIC": "COSINE"
        }
        ),
        NumericField("rating"),
        TagField("genre"),
        TextField("title")
)

try:
    client.ft(index_name).info()
    print("Index exists!")
except:
    # index Definition
    definition = IndexDefinition(index_type=IndexType.HASH)

    # create Index
    client.ft(index_name).create_index(fields=schema, definition=definition)

Index exists!


## Populate index

In [21]:
def load_docs(client: Redis, data: list[dict]):
    for i, d in enumerate(data):
        client.hset(
            i,
            mapping = d
        )

def print_results(res):
    docs = [(doc.title, doc.genre, doc.rating) for doc in res.docs]
    print(f"Top {len(docs)} movies: ", docs)

In [ ]:
load_docs(client, movie_data)

In [14]:
res = client.ft(index_name).search("*")
res

Result{20 total, docs: [Document {'id': '0', 'payload': None, 'genre': 'action', 'vector': '\x00\x00\x00`ӌ?\x00\x00\x00\x01La?\x00\x00\x00@Dv?\x00\x00\x00gٯ\x00\x00\x00@y\x00\x00\x00`B?\x00\x00\x00?\x00\x00\x00ݮ?\x00\x00\x00L\x00\x00\x00"?\x00\x00\x00\x00\x00?\x00\x00\x00g\x00\x00\x00R\x1e\x00\x00\x00@̘?\x00\x00\x00;?\x00\x00\x00@\x0fǂ\x00\x00\x00\x00z?\x00\x00\x00G?\x00\x00\x00=?\x00\x00\x00\x17?\x00\x00\x00@P\x00\x00\x00`ka\x00\x00\x00\x00I?\x00\x00\x00\x00ힿ\x00\x00\x00¿\x00\x00\x00`D \x00\x00\x00 ?\x00\x00\x00\x00\x16\\L\x00\x00\x00`\x14\x00\x00\x00\x00\x00\x00\x00@j?\x00\x00\x00\\n\x00\x00\x00?\x00\x00\x00\x07?\x00\x00\x00\rH\x00\x00\x00@F\x0b\x00\x00\x00`d\x00\x00\x00 \x00\x00\x00@\x1b?\x00\x00\x00@Ĉ?\x00\x00\x00X?\x00\x00\x00\x00\x12\x00\x00\x00`Y\x0b?\x00\x00\x00 at?\x00\x00\x00l\x00\x00\x00\x00\x00\x00~?\x00\x00\x00^?\x00\x00\x00?\x00\x00\x00\x00\x00\x00-\t\x00\x00\x00@ɢw?\x00\x00\x00`\ue721\x00\x00\x00\x00d&\x00\x00\x00\x10\x7f\x00\x00\x00פn\x00\x00\x00Q?\x00\x00\x00\x00P?\x00

## Index loaded now we can perform vector search

### basic vector search

In [22]:
from redis.commands.search.query import Query

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query('(*)=>[KNN 3 @vector $vec_param AS dist]').sort_by('dist')

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)


Top 3 movies:  [('Fast & Furious 9', 'action', '6'), ('Despicable Me', 'comedy', '7'), ('The Incredibles', 'comedy', '8')]


### Hybrid filter vector search

Redis allows you to combine filter searches on fields within the index object allowing us to create more specific searches.

In [23]:
# Search for top 3 movies specifically in the action genre

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: genre is a tag field in our schema so the syntax is @<field_name>:{ <tag> | <tag> | ...}
query = Query(f'(@genre:{{action}})=>[KNN 3 @vector $vec_param AS dist]').sort_by('dist')

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Fast & Furious 9', 'action', '6'), ('Mad Max: Fury Road', 'action', '8'), ('Explosive Pursuit', 'action', '7')]


In [24]:
# Search for top 3 movies specifically in the action genre with ratings at or above a 7

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query(f'(@genre:{{action}} & (@rating:[7 inf]))=>[KNN 3 @vector $vec_param AS dist]').sort_by('dist')

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Mad Max: Fury Road', 'action', '8'), ('Explosive Pursuit', 'action', '7'), ('The Avengers', 'action', '8')]


## Range queries

Range queries allow you to set a pre defined "threshold" for which we want to return documents. This is helpful when you only want documents with a certain distance from the search query.

In [33]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

query = (
    Query("@vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("title", "rating", "genre", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vector": embedded_user_query
}

res = client.ft(index_name).search(query, query_params)
print_results(res)


Top 6 movies:  [('The Incredibles', 'comedy', '8'), ('Black Widow', 'action', '7'), ('Despicable Me', 'comedy', '7'), ('Shrek', 'comedy', '8'), ('Monsters, Inc.', 'comedy', '8'), ('Aladdin', 'comedy', '8')]


Like the queries above, we can also chain additional filters and conditional operators with range queries. The following adds an `or` condition that returns vector search within the defined range or with a rating at or above 9.

In [48]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

query = (
    Query("@rating:[9 +inf] | @vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("title", "rating", "genre", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.7,
    "vector": embedded_user_query
}

res = client.ft(index_name).search(query, query_params)
print_results(res)

Top 3 movies:  [('The Incredibles', 'comedy', '8'), ('The Dark Knight', 'action', '9'), ('Inception', 'action', '9')]
